In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

def get_optimized_model_v1(device='cpu'):
    """
    Modèle VGG11 optimisé avec kernels 5x5 et pooling adaptatif
    Réduction des paramètres et meilleure préservation des features spatiales
    """
    model = nn.Sequential(
        # Bloc 1
        nn.Conv2d(3, 32, kernel_size=5, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 32, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.SpatialDropout2d(0.25) if hasattr(nn, 'SpatialDropout2d') else nn.Dropout2d(0.25),
        nn.MaxPool2d(2),

        # Bloc 2
        nn.Conv2d(32, 64, kernel_size=5, padding=2),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size=5, padding=2),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.SpatialDropout2d(0.25) if hasattr(nn, 'SpatialDropout2d') else nn.Dropout2d(0.25),
        nn.MaxPool2d(2),

        # Bloc 3
        nn.Conv2d(64, 128, kernel_size=5, padding=2),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.SpatialDropout2d(0.25) if hasattr(nn, 'SpatialDropout2d') else nn.Dropout2d(0.25),
        nn.MaxPool2d(2),

        # Bloc 4
        nn.Conv2d(128, 128, kernel_size=5, padding=2),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.SpatialDropout2d(0.25) if hasattr(nn, 'SpatialDropout2d') else nn.Dropout2d(0.25),
        nn.MaxPool2d(2),

        # INNOVATION: Adaptive pooling pour taille fixe
        nn.AdaptiveAvgPool2d((4, 4)),

        # Classificateur compact progressif
        nn.Flatten(),  # 128 * 4 * 4 = 2048
        nn.Linear(2048, 1024),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 10)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    return model, loss_fn, optimizer


In [3]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torchsummary import summary

# Dataset CIFAR-10 corrigé
class CIFAR10Dataset(Dataset):
    def __init__(self, images, targets, device='cpu'):
        # Conversion correcte pour CIFAR-10
        if isinstance(images, torch.Tensor):
            images = images.float() / 255.0
        else:
            images = torch.tensor(images, dtype=torch.float32) / 255.0
        
        if isinstance(targets, torch.Tensor):
            targets = targets.long()
        else:
            targets = torch.tensor(targets, dtype=torch.long)

        # CIFAR-10: (N, H, W, C) -> (N, C, H, W)
        if images.ndim == 4:
            images = images.permute(0, 3, 1, 2)
        
        self.x = images
        self.y = targets
        self.device = device

    def __getitem__(self, ix):
        x = self.x[ix].to(self.device)
        y = self.y[ix].to(self.device)
        return x, y

    def __len__(self):
        return len(self.x)

# Classe pour convolution séparable
class DepthwiseSeparableConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=4, padding=2):
        super().__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels,
                                 kernel_size=kernel_size, padding=padding,
                                 groups=in_channels, bias=False)
        self.pointwise = nn.Conv2d(in_channels, out_channels,
                                 kernel_size=1, bias=False)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

# Modèles corrigés


def get_optimized_model_v3(device='cpu'):
    model = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=4, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Dropout2d(0.25),

        DepthwiseSeparableConv2d(32, 64, kernel_size=4, padding=2),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d((6, 6)),
        nn.Dropout2d(0.3),

        nn.Flatten(),
        nn.Linear(2304, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 10)
    ).to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

# Fonctions d'entraînement corrigées
def train_batch(x, y, model, optimizer, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

@torch.no_grad()
def val_loss(x, y, model, loss_fn):
    model.eval()
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

def get_data():
    # Chargement CIFAR-10
    data_folder = './data'
    train_dataset = datasets.CIFAR10(data_folder, download=True, train=True)
    val_dataset = datasets.CIFAR10(data_folder, download=True, train=False)
    
    tr_images = train_dataset.data
    tr_targets = train_dataset.targets
    val_images = val_dataset.data
    val_targets = val_dataset.targets
    
    train = CIFAR10Dataset(tr_images, tr_targets)
    trn_dl = DataLoader(train, batch_size=32, shuffle=True)
    val = CIFAR10Dataset(val_images, val_targets)
    val_dl = DataLoader(val, batch_size=len(val_images), shuffle=False)
    
    return trn_dl, val_dl

# Code principal d'exécution
if __name__ == "__main__":
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Utilisation du device: {device}")
    
    # Charger les données
    trn_dl, val_dl = get_data()
    
    # Initialiser le modèle
    model, loss_fn, optimizer = get_optimized_model_v1(device=device)
    
    # Afficher le résumé
    print("\nRésumé du modèle:")
    summary(model, (3, 32, 32))
    
    # Entraînement
    train_losses, train_accuracies = [], []
    val_losses, val_accuracies = [], []
    
    for epoch in range(5):
        print(f"\nEpoch {epoch+1}/5")
        
        # Entraînement
        train_epoch_losses, train_epoch_accuracies = [], []
        for ix, batch in enumerate(trn_dl):
            x, y = batch
            batch_loss = train_batch(x, y, model, optimizer, loss_fn)
            train_epoch_losses.append(batch_loss)
        
        # Précision d'entraînement
        for ix, batch in enumerate(trn_dl):
            x, y = batch
            is_correct = accuracy(x, y, model)
            train_epoch_accuracies.extend(is_correct)
        
        # Validation
        for ix, batch in enumerate(val_dl):
            x, y = batch
            val_is_correct = accuracy(x, y, model)
            validation_loss = val_loss(x, y, model, loss_fn)
        
        train_epoch_loss = np.mean(train_epoch_losses)
        train_epoch_accuracy = np.mean(train_epoch_accuracies)
        val_epoch_accuracy = np.mean(val_is_correct)
        
        train_losses.append(train_epoch_loss)
        train_accuracies.append(train_epoch_accuracy)
        val_losses.append(validation_loss)
        val_accuracies.append(val_epoch_accuracy)
        
        print(f"Loss: {train_epoch_loss:.4f}, Acc: {train_epoch_accuracy:.4f}")
        print(f"Val Loss: {validation_loss:.4f}, Val Acc: {val_epoch_accuracy:.4f}")

RuntimeError: operator torchvision::nms does not exist